## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [6]:
import pandas as pd
import os
from xlrd import *
import csv 
import sys
import win32com.client as w3c
from tempfile import NamedTemporaryFile

In [2]:
PATH = 'C:/Users/smsoh/Documents/GitHub/CLEF2021/task3/data/Task3a_Batch_1.xlsx'
PASS = 'fake@2021'

In [3]:
xlapp = w3c.Dispatch("Excel.Application")
xlwb = xlapp.Workbooks.Open(PATH, False, True, None, PASS)

In [4]:
#df = pd.read_csv("data/Task3a_Batch_1.xlsx")
f = NamedTemporaryFile(delete=False, suffix='.csv')
f.close()
os.unlink(f.name)

xlCSVWindows = 0x17
xlwb.SaveAs(Filename=f.name, FileFormat=xlCSVWindows)
df = pd.read_csv(f.name)
df

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false
...,...,...,...,...
432,e01f532a,Rosemary Frei and Patrick Corbett A high-prof...,�No one has died from the coronavirus�,false
433,3c791c29,As the coronavirus pandemic continues to devas...,Why Did 21 Million Phone Numbers Disappear fro...,false
434,fe6a9262,Belgium Health Minister Maggie de Block has pu...,Belgium Health Minister puts ban on non-essent...,false
435,97113f55,"Responding To Backlash, Netflix Clarifies Its ...","Responding To Backlash, Netflix Clarifies Its ...",false


In [5]:
df.head()

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false


In [8]:
df['label'] = df['our rating'].apply(lambda x: 0 if 'true' in x.lower() else ( 1 if 'partially false' in x.lower() else ( 2 if 'false' in x.lower() else 3)))
df

,public_id,text,title,our rating,label
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false,2
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false,1
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false,1
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false,2
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false,2
...,...,...,...,...,...
432,e01f532a,Rosemary Frei and Patrick Corbett A high-prof...,�No one has died from the coronavirus�,false,2
433,3c791c29,As the coronavirus pandemic continues to devas...,Why Did 21 Million Phone Numbers Disappear fro...,false,2
434,fe6a9262,Belgium Health Minister Maggie de Block has pu...,Belgium Health Minister puts ban on non-essent...,false,2
435,97113f55,"Responding To Backlash, Netflix Clarifies Its ...","Responding To Backlash, Netflix Clarifies Its ...",false,2


In [9]:
###Drop Nan Values
df=df.dropna()


In [7]:
df_true = pd.read_csv("data/True.csv")
df_true = df_true.drop([ "title","subject","date"], axis = 1)
df_true['label'] = 1
df_fake = pd.read_csv("data/Fake.csv")
df_fake = df_fake.drop([ "title","subject","date"], axis = 1)
df_fake['label'] = 0
df = pd.concat([df_fake, df_true], axis=0)
df.sort_index(inplace=True)
df

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
...,...,...
23476,21st Century Wire says As 21WIRE reported earl...,0
23477,21st Century Wire says It s a familiar theme. ...,0
23478,Patrick Henningsen 21st Century WireRemember ...,0
23479,21st Century Wire says Al Jazeera America will...,0


In [8]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [9]:
## Get the Dependent features
y=df['label']

In [10]:
X.shape

(44898, 1)

In [11]:
y.shape

(44898,)

In [12]:
import tensorflow as tf

In [15]:
tf.__version__

'2.4.1'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [15]:
messages=X.copy()

In [17]:
messages['text'][1]

1    WASHINGTON (Reuters) - Transgender people will...
1    House Intelligence Committee Chairman Devin Nu...
Name: text, dtype: object

In [18]:
messages.reset_index(inplace=True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\smsoh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

4
41565
41566
41567
41568
41569
41570
41571
41572
41573
41574
41575
41576
41577
41578
41579
41580
41581
41582
41583
41584
41585
41586
41587
41588
41589
41590
41591
41592
41593
41594
41595
41596
41597
41598
41599
41600
41601
41602
41603
41604
41605
41606
41607
41608
41609
41610
41611
41612
41613
41614
41615
41616
41617
41618
41619
41620
41621
41622
41623
41624
41625
41626
41627
41628
41629
41630
41631
41632
41633
41634
41635
41636
41637
41638
41639
41640
41641
41642
41643
41644
41645
41646
41647
41648
41649
41650
41651
41652
41653
41654
41655
41656
41657
41658
41659
41660
41661
41662
41663
41664
41665
41666
41667
41668
41669
41670
41671
41672
41673
41674
41675
41676
41677
41678
41679
41680
41681
41682
41683
41684
41685
41686
41687
41688
41689
41690
41691
41692
41693
41694
41695
41696
41697
41698
41699
41700
41701
41702
41703
41704
41705
41706
41707
41708
41709
41710
41711
41712
41713
41714
41715
41716
41717
41718
41719
41720
41721
41722
41723
41724
41725
41726
41727
41728
41729
41730
41

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

### Embedding Representation

In [38]:
sent_length=1000
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3904 3890 3924]
 [   0    0    0 ...  329 4814  239]
 [   0    0    0 ... 1582 4219 1748]
 ...
 [4998 2635 4654 ... 4128 4081 1044]
 [   0    0    0 ... 4128 2568 1796]
 [   0    0    0 ... 4128 3411 1044]]


In [39]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [40]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 40)          200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
len(embedded_docs),y.shape

(44898, (44898,))

In [42]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [43]:
X_final.shape,y_final.shape

((44898, 1000), (44898,))

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=8)

### Model Training

In [45]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
471/471 [==============================] - 1139s 2s/step - loss: 0.3239 - accuracy: 0.8551 - val_loss: 0.1008 - val_accuracy: 0.9719
Epoch 2/10
471/471 [==============================] - 1093s 2s/step - loss: 0.0770 - accuracy: 0.9767 - val_loss: 0.0491 - val_accuracy: 0.9864
Epoch 3/10
471/471 [==============================] - 1186s 3s/step - loss: 0.0385 - accuracy: 0.9876 - val_loss: 0.0497 - val_accuracy: 0.9861
Epoch 4/10
471/471 [==============================] - 1213s 3s/step - loss: 0.1298 - accuracy: 0.9454 - val_loss: 0.0704 - val_accuracy: 0.9792
Epoch 5/10
471/471 [==============================] - 1317s 3s/step - loss: 0.0341 - accuracy: 0.9901 - val_loss: 0.0327 - val_accuracy: 0.9912
Epoch 6/10
471/471 [==============================] - 1274s 3s/step - loss: 0.0290 - accuracy: 0.9903 - val_loss: 0.0409 - val_accuracy: 0.9890
Epoch 7/10
471/471 [==============================] - 1212s 3s/step - loss: 0.0173 - accuracy: 0.9959 - val_loss: 0.0653 - val_accuracy:

### Adding Dropout 

In [46]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [47]:
y_pred=model.predict_classes(X_test)

C:\Users\smsoh\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [48]:
from sklearn.metrics import confusion_matrix

In [49]:
confusion_matrix(y_test,y_pred)

array([[4528, 3215],
       [3593, 3481]], dtype=int64)

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5405277721536074